In [256]:
import qcfractal.interface as ptl
client = ptl.FractalClient(address="localhost:7777", verify=False)

# 1. Create a collection (OptmizationDataset)

first create a collection that will contain all of your small molecules 

In [8]:
ds_mols = ptl.collections.OptimizationDataset("project_molecules", client=client)
ds_mols.save()

KeyError: 'Error adding collection: \n(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_collection_lname"\nDETAIL:  Key (collection, lname)=(optimizationdataset, project_molecules) already exists.\n\n[SQL: INSERT INTO collection (collection_type, collection, lname, name, tags, tagline, description, "group", visibility, view_url_hdf5, view_url_plaintext, view_metadata, view_available, provenance, extra) VALUES (%(collection_type)s, %(collection)s, %(lname)s, %(name)s, %(tags)s, %(tagline)s, %(description)s, %(group)s, %(visibility)s, %(view_url_hdf5)s, %(view_url_plaintext)s, %(view_metadata)s, %(view_available)s, %(provenance)s, %(extra)s) RETURNING collection.id]\n[parameters: {\'collection_type\': None, \'collection\': \'optimizationdataset\', \'lname\': \'project_molecules\', \'name\': \'project_molecules\', \'tags\': \'[]\', \'tagline\': None, \'description\': None, \'group\': \'default\', \'visibility\': True, \'view_url_hdf5\': None, \'view_url_

# 2. load the collection beforehand 

In [209]:
ds_mols = client.get_collection('OptimizationDataset', 'radicals')

TypeError: get_collection() missing 1 required positional argument: 'name'

 # 3. create a molecule xyz and an object

now add a small molecule eg water to the collection

In [11]:
h2o_xyz = '''O          0.00000        0.00000        0.11765
    H          0.00000        0.76012       -0.47058
    H          0.00000       -0.76012       -0.47058
'''

In [12]:
mol_h2o = ptl.Molecule.from_data(h2o_xyz)

visualise

In [13]:
mol_h2o

NGLWidget()

# 4. Add water to your project_moelecules collection:

In [14]:
ds_mols.add_entry("h2o", mol_h2o)
ds_mols.save()

KeyError: 'Record h2o already in the dataset.'

# 5. Send the geometry optimsation for water 

first add the specifation

In [15]:
add_spec = {'name': 'blyp-d3bj_def2-svpd',
        'description': 'Geometric + Psi4/blyp-d3bj/def2-svpd.',
        'optimization_spec': {'program': 'geometric', 'keywords': None},
        'qc_spec': {'driver': 'gradient',
        'method': 'blyp-d3bj',
        'basis': 'def2-svpd',
        'keywords': None,
        'program': 'psi4'}}
ds_mols.add_specification(**add_spec,overwrite=True)
ds_mols.save()

'1028'

here you specify the level of theory and basis set, play around and save a few specifications

In [48]:
ds_mols.compute('mpwb1k-d3bj_def2-tzvp', tag='basic_small')

0

run the following to see if the the water molecules has computed 

In [347]:
ds_mols  = client.get_collection("OptimizationDataset", "project_molecules")
ds_mols.status(collapse = False)

,blyp-d3bj_def2-svpd,b3lyp-d3bj_def2-svpd,hf-3c-d3bj_MINIX,b3lyp_def2-svpd,hf-3c_MINIX,hf-d3bj_sto-3g,hf_sto-3g,hf_MINIX,mpwb1k-d3bj_def2-tzvp,mpwb1k-d3bj_def2-svpd
C2H2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2_CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2O_H2O,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2O,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2_CO2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CH4,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CH3OH,None,INCOMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE


In [348]:
new_tag = "m_project"

#for spec in ds_mols.df.columns:
#for entry in ds_mols.df.index:
rr = ds_mols.get_record('NHCH2','mpwb1k-d3bj_def2-svpd')
ct =  client.modify_tasks("s", new_tag=new_tag, base_result=rr.id)
#cr =  client.modify_tasks("restart", base_result=rr.id)
#print('NH2','mpwb1k-d3bj_def2-svpd',ct,cr)

ValueError: Operation 's' is not available, valid operations are: {'restart', 'regenerate', 'modify'}

In [71]:
ds_mols.compute('mpwb1k-d3bj_def2-tzvp', tag=new_tag,subset={'NH2'})

1

# 6. Now follow this workflow for your additonal molecules

CH3OH, CO2, CO, H2CO 

make the xyz files on avogadro and put those into the workbook and follow the wokrflow

Bonus points if you make a loop for this workflow #nopressure

## Creating a class of the workflow

In [28]:
class workflow():
    
    def __init__(self,dataset,molecule,name):
        self.dataset = client.get_collection('OptimizationDataset','project_molecules')
        self.molecule=molecule
        self.xyz_file = f'/home/fastdata/Shared/Molecules_AL/{molecule}.xyz' # as in the command line
        self.name = name
        
    def create_molecule(self):
        with open(self.xyz_file,'r') as file:
            lines=file.readlines()[2:]
            lines=[' '.join(line.split()) for line in lines] # clean each line by stripping extra speaces and newlines and join them with a single space   
            lines = ''+ '\n'.join(lines)+''
            lines=str(lines)
        mol = ptl.Molecule.from_data(lines)
        return mol
        
    def add_specification(self):
        self.specification={'name':self.name, 
        'description':f'Geometric +Psi4/{self.name.split("_")[0]}/{self.name.split("_")[1]}.',
        'optimization_spec':{'program':'geometric','keywords':None},
        'qc_spec':{'driver':'gradient',
        'method':self.name.split('_')[0],
        'basis':self.name.split('_')[1],
        'keywords':None,
        'program': 'psi4'}}
            
        return self.specification
        
        
    def compute_molecule(self):
        #deleting the molecule if it exists
        if self.molecule.lower() in self.dataset.data.records:
            del self.dataset.data.records[f"{self.molecule.lower()}"]
            self.dataset.save()
            
        self.dataset.add_entry(f"{self.molecule}",
                                workflow(self.dataset,self.molecule,self.name).create_molecule())
            
        self.dataset.save()
        #adding the specification if not exists
        if workflow(self.dataset,self.molecule,self.name).add_specification()['name'].lower() not in self.dataset.data.specs:
            self.dataset.add_specification(**workflow(self.dataset,self.molecule,self.name).add_specification(),overwrite=True)
            self.dataset.save()
        
        self.dataset.compute(workflow(self.dataset,self.molecule,self.name).add_specification()['name'],tag='basic_small',
                            subset={f"{self.molecule}"})
        self.dataset=client.get_collection('OptimizationDataset','project_molecules')
        status=self.dataset.status(collapse=False)
            
        return status
        
    def check_status_only(self):
        return self.dataset.status(collapse=False)
            
    def get_results(self):
        mol_results= self.dataset.get_record(f'{self.molecule}',workflow(self.dataset,self.molecule,self.name).add_specification()['name'])
        mol_results_dict = mol_results.dict()
        return mol_results_dict,mol_results
            
            
            
CO=workflow(ds_mols,'CO','blyp-d3bj_def2-svpd')


            
            
            
        

In [56]:
molecules=['CO','CH3OH','CO2','H2CO','CH2OH','CH3O','CH3','H2S','H2',
           'HCl','HCN','HCOOH','HCO','HF','HNC','N2','NH2','NH3','NHCH2','OH']
# list_of_mol_objects=[]

# for molecule in molecules:
optimisation=workflow(ds_mols,'NH2','mpwb1k-d3bj_def2-svpd')
mol=optimisation.create_molecule()
#list_of_mol_objects.append(mol)
optimisation.add_specification()
optimisation.compute_molecule()
   
     #optimisation.get_results()[1].show_history()
    #print(optimisation.get_results()[1].energies)
    
    
    

#mol_visualisation_dict={mol:object for mol,object in zip(molecules,list_of_mol_objects)}
    
#mol_visualisation_dict['NHCH2']
#workflow(ds_mols,molecule,'mpwb1k-d3bj_def2-tzvp').check_status_only()


    




























,blyp-d3bj_def2-svpd,b3lyp-d3bj_def2-svpd,hf-3c-d3bj_MINIX,b3lyp_def2-svpd,hf-3c_MINIX,hf-d3bj_sto-3g,hf_sto-3g,hf_MINIX,mpwb1k-d3bj_def2-tzvp,mpwb1k-d3bj_def2-svpd
C2H2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2_CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2O_H2O,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2O,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2_CO2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CH4,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CH3OH,None,INCOMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE


,blyp-d3bj_def2-svp,mpwb1k-d3bj_def2-tzvp,b3lyp-d3bj_def2-tzvp,revpbe0-d3bj_def2-tzvp,blyp-d3bj_def2-svpd,hf3c_minix
ch3oh,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE


In [345]:
optimisation.check_status_only()

,blyp-d3bj_def2-svpd,b3lyp-d3bj_def2-svpd,hf-3c-d3bj_MINIX,b3lyp_def2-svpd,hf-3c_MINIX,hf-d3bj_sto-3g,hf_sto-3g,hf_MINIX,mpwb1k-d3bj_def2-tzvp,mpwb1k-d3bj_def2-svpd
C2H2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2_CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2O_H2O,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2O,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2_CO2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CH4,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CH3OH,None,INCOMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE
H2CO,None,COMPLETE,None,None,None,None,None,None,COMPLETE,COMPLETE


## For NH2

In [341]:
ds_rads = client.get_collection('OptimizationDataset', 'radicals')
ds_rads.status(collapse=False)

,blyp-d3bj_def2-svp,mpwb1k-d3bj_def2-tzvp,ublyp-d3bj_def2-svp,umpwb1k-d3bj_def2-tzvp,ublyp-d3bj_def2-svpd
ch2o,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
ch3,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
ch2,ERROR,COMPLETE,ERROR,COMPLETE,INCOMPLETE
ch,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
c2h2,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
hco,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
oh,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
ch3o,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
ch2oh,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
h2,INCOMPLETE,INCOMPLETE,INCOMPLETE,INCOMPLETE,INCOMPLETE


In [328]:
# del ds_rads.data.records["nh2"]
# ds_rads.save()

del ds_rads.data.specs['um06-2x_def2-svpd']
ds_rads.save()

'60'

In [330]:
# nh2_xyz = '''N       -0.8261538073      1.4306939924      0.2915485169                 
# H       -1.5491348976      0.7032945644      0.3209494281                 
# H       -0.4652970341      1.4876797705     -0.6672064106                           
# '''

# Try H2 to see if it works 

h2_xyz = '''H       -1.5641988998      3.8436699361      0.0000000000                 
H       -0.8573911002      3.8026000639      0.0000000000   '''

In [331]:
mol_h2 = ptl.Molecule.from_data(h2_xyz)
ds_rads.add_entry("h2", mol_h2)
ds_rads.save()




'60'

In [333]:
for spec in ds_rads.df.columns:
     ds_rads.compute(spec, tag='basic_small',subset={'h2'})

In [223]:
mol_nh2 = ptl.Molecule.from_data(nh2_xyz)

In [224]:
mol_nh2

NGLWidget()

In [227]:
ds_mols.add_entry("nh2", mol_nh2)
ds_mols.save()

'60'

## For NHCH2

In [225]:
nhch2_xyz = '''C       -1.0480188785      1.5576879022      0.1590231052                 
N       -0.6829024196      0.3673843635      0.4841193986                 
H       -0.8210353554      2.4130699678      0.8146706657                 
H       -1.5945011742      1.7635647955     -0.7755106431                 
H       -0.9758821725     -0.3008170290     -0.2382425264                 
'''

In [226]:
mol_nhch2 = ptl.Molecule.from_data(nhch2_xyz)
mol_nhch2

NGLWidget()

In [228]:
ds_mols.add_entry("nhch2", mol_nhch2)
ds_mols.save()

'60'

In [230]:
for spec in ds_mols.df.columns:
     ds_mols.compute(spec, tag='basic_small',subset={'nh2','nhch2'})

In [326]:

add_spec = {'name': 'um06-2x_def2-svpd',
        'description': 'Geometric + Psi4/um06-2x/def2-svpd.',
        'optimization_spec': {'program': 'geometric', 'keywords': None},
        'qc_spec': {'driver': 'gradient',
        'method': 'um06-2x',
        'basis': 'def2-svpd',
        'keywords': None,
        'program': 'psi4'}}
ds_rads.add_specification(**add_spec,overwrite=True)
ds_rads.save()

'60'

In [303]:
ds_mols.compute('um06-2x_def2-svpd', tag='basic_small')

0

## For CO2 surfaces

In [343]:
ds_co2  = client.get_collection("OptimizationDataset", "co2_beep")
ds_co2.status(collapse = False)

,mpwb1k-d3bj_def2-tzvp,mpwb1k-d3bj_def2-svpd,um06-2x_def2-svpd
CD2_01,COMPLETE,COMPLETE,INCOMPLETE
CD3_01,COMPLETE,COMPLETE,INCOMPLETE
CD3_02,COMPLETE,COMPLETE,INCOMPLETE
CD1_01,COMPLETE,COMPLETE,INCOMPLETE


In [305]:
add_spec = {'name': 'um06-2x_def2-svpd',
        'description': 'Geometric + Psi4/um06-2x/def2-svpd.',
        'optimization_spec': {'program': 'geometric', 'keywords': None},
        'qc_spec': {'driver': 'gradient',
        'method': 'um06-2x',
        'basis': 'def2-svpd',
        'keywords': None,
        'program': 'psi4'}}
ds_co2.add_specification(**add_spec,overwrite=True)
ds_co2.save()

'1005'

In [297]:
new_tag = "co2_surf_5"

#for spec in ds_mols.df.columns:
for entry in ds_co2.df.index:
    rr = ds_co2.get_record(entry,'um06-2x_def2-svpd')
    cr =  client.modify_tasks("restart", base_result=rr.id)
    ct =  client.modify_tasks("modify", new_tag=new_tag, base_result=rr.id)
    print(entry,'um06-2x_def2-svpd',ct,cr)

ValueError: Operation 'e' is not available, valid operations are: {'restart', 'regenerate', 'modify'}

In [306]:
ds_co2.compute('um06-2x_def2-svpd', tag='co2_surf')

0

In [213]:
# add uM06-2X/def2-TZVP  to co2_beep collection

add_spec = {'name': 'um06-2x_def2-svpd',
        'description': 'Geometric + Psi4/um06-2x/def2-svpd.',
        'optimization_spec': {'program': 'geometric', 'keywords': None},
        'qc_spec': {'driver': 'gradient',
        'method': 'um06-2x',
        'basis': 'def2-svpd',
        'keywords': None,
        'program': 'psi4'}}
ds_co2.add_specification(**add_spec,overwrite=True)
ds_co2.save()



'1005'

In [300]:
# del ds_co2.data.specs['um06-2x_def2-svpd']
# ds_co2.save()

'1005'

In [349]:
ds_co2  = client.get_collection("OptimizationDataset", "co2_beep")
ds_co2.status(collapse = False)

,mpwb1k-d3bj_def2-tzvp,mpwb1k-d3bj_def2-svpd,um06-2x_def2-svpd
CD2_01,COMPLETE,COMPLETE,INCOMPLETE
CD3_01,COMPLETE,COMPLETE,INCOMPLETE
CD3_02,COMPLETE,COMPLETE,INCOMPLETE
CD1_01,COMPLETE,COMPLETE,INCOMPLETE
